# Потребление алкоголя, которое регистрируется в литрах чистого алкоголя на душу населения в возрасте 15+ лет

In [42]:
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке:

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/alcohol-recorded-per-capita-(15-)-consumption-(in-litres-of-pure-alcohol)

In [43]:
# Загрузим таблицу
data = pd.read_csv('../data/who_alcohol.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",numeric,EMR,Eastern Mediterranean,Country,AFG,Afghanistan,Year,2020,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,EN,2024-08-11T21:00:00.000Z
1,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",numeric,EMR,Eastern Mediterranean,Country,KWT,Kuwait,Year,2020,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,EN,2024-08-11T21:00:00.000Z


Описание нужных нам столбцов.

- `SpatialDimValueCode` - код страны
- `Location` - название страны
- `Period` - год
- `FactValueNumeric` - потребление алкоголя на душу населения (15+) (в литрах чистого алкоголя)

Оставим только их.

In [44]:
data = data[['SpatialDimValueCode', 'Location', 'Period', 'FactValueNumeric']]

# Переименуем столбец FactValueNumeric в AlcoholСonsumption
data.rename(
    columns={'FactValueNumeric': 'AlcoholСonsumption'},
    inplace=True,
)

data.head()

,SpatialDimValueCode,Location,Period,AlcoholСonsumption
0,AFG,Afghanistan,2020,0.009
1,KWT,Kuwait,2020,0.000
2,LBY,Libya,2020,0.011
3,MRT,Mauritania,2020,0.000
4,PAK,Pakistan,2020,0.048


In [45]:
# Убедимся, что нет пропусков
data.isna().sum().sum()

0

В таблице пропущены значения для некоторых лет для стран *Serbia* (Сербия) и *Montenegro* (Черногория).

### Заполнение пропусков для страны *Serbia* (Сербия)

Посмотрим, за какие годы нет данных для [Сербии](https://ru.wikipedia.org/wiki/%D0%A1%D0%B5%D1%80%D0%B1%D0%B8%D1%8F).

In [46]:
mask = data['Location'] == 'Serbia'

data[mask].sort_values(by='Period')

,SpatialDimValueCode,Location,Period,AlcoholСonsumption
2825,SRB,Serbia,2006,8.82
2640,SRB,Serbia,2007,9.42
2453,SRB,Serbia,2008,9.59
2265,SRB,Serbia,2009,9.78
2064,SRB,Serbia,2010,8.87
1871,SRB,Serbia,2011,8.55
1683,SRB,Serbia,2012,8.78
1485,SRB,Serbia,2013,7.94
1284,SRB,Serbia,2014,7.19
1097,SRB,Serbia,2015,7.20


Видим, что нет данных за 2000-2005 гг. и за 2020 г.

Заполним значения для Сербии средним по окружающим ее странам:

![](../images/map_serbia.jpg)

- Венгрия (*Hungary*)
- Румыния (*Romania*)
- Болгария (*Bulgaria*)
- Северная Македония (*North Macedonia*)
- Албания (*Albania*)
- Черногория (*Montenegro*) - нет данных за те же годы, что и для Сербии
- Босния и Герцеговина (*Bosnia and Herzegovina*)
- Хорватия (*Croatia*)

In [47]:
# Маска по странам
mask_locations = data['Location'].isin([
    'Hungary', 'Romania', 'Bulgaria', 'North Macedonia',
    'Albania', 'Bosnia and Herzegovina', 'Croatia'
])
# Маска по годам
mask_years = data['Period'].isin([
    2000, 2001, 2002, 2003, 2004, 2005, 2020,
])

# Сгруппируем данные соседних стран по году и возьмем среднее
serbia_data = data[mask_locations & mask_years] \
    .groupby(by='Period')[['AlcoholСonsumption']].agg('mean').reset_index()

# Заполним код и название страны
serbia_data['SpatialDimValueCode'] = 'SRB'
serbia_data['Location'] = 'Serbia'
# Переставим столбцы в том же порядке, как у основной таблицы
serbia_data = serbia_data.reindex(columns=list(data.columns))
# Посмотрим на результат
serbia_data

,SpatialDimValueCode,Location,Period,AlcoholСonsumption
0,SRB,Serbia,2000,8.495714
1,SRB,Serbia,2001,8.495714
2,SRB,Serbia,2002,8.757143
3,SRB,Serbia,2003,8.694286
4,SRB,Serbia,2004,8.432857
5,SRB,Serbia,2005,8.268571
6,SRB,Serbia,2020,7.622857


In [48]:
# Объединим данные для Сербии с основной таблицей
data = pd.concat([
    data,
    serbia_data,
], ignore_index=True)

### Заполнение пропусков для страны *Montenegro* (Черногория)

Посмотрим, за какие годы нет данных для [Черногории](https://ru.wikipedia.org/wiki/%D0%A7%D0%B5%D1%80%D0%BD%D0%BE%D0%B3%D0%BE%D1%80%D0%B8%D1%8F).

In [49]:
mask = data['Location'] == 'Montenegro'

data[mask].sort_values(by='Period')

,SpatialDimValueCode,Location,Period,AlcoholСonsumption
2712,MNE,Montenegro,2006,10.38
2643,MNE,Montenegro,2007,9.68
2341,MNE,Montenegro,2008,10.89
2139,MNE,Montenegro,2009,10.17
2076,MNE,Montenegro,2010,9.94
1877,MNE,Montenegro,2011,9.02
1690,MNE,Montenegro,2012,9.37
1506,MNE,Montenegro,2013,9.62
1305,MNE,Montenegro,2014,8.55
1116,MNE,Montenegro,2015,8.49


Видим, что нет данных за 2000-2005 гг. и за 2020 г.

Заполним значения для Черногории средним по окружающим ее странам:

<img src="../images/map_montenegro.jpg" width="400" />

- Хорватия (*Croatia*)
- Босния и Герцеговина (*Bosnia and Herzegovina*)
- Сербия (Serbia)
- Албания (*Albania*)

In [50]:
# Маска по странам
mask_locations = data['Location'].isin([
    'Croatia', 'Bosnia and Herzegovina', 'Serbia', 'Albania',
])
# Маска по годам
mask_years = data['Period'].isin([
    2000, 2001, 2002, 2003, 2004, 2005, 2020,
])

# Сгруппируем данные соседних стран по году и возьмем среднее
montenegro_data = data[mask_locations & mask_years] \
    .groupby(by='Period')[['AlcoholСonsumption']].agg('mean').reset_index()
# Заполним код и название страны
montenegro_data['SpatialDimValueCode'] = 'MNE'
montenegro_data['Location'] = 'Montenegro'
# Переставим столбцы в том же порядке, как у основной таблицы
montenegro_data = montenegro_data.reindex(columns=list(data.columns))
# Посмотрим на результат
montenegro_data

,SpatialDimValueCode,Location,Period,AlcoholСonsumption
0,MNE,Montenegro,2000,7.291429
1,MNE,Montenegro,2001,7.291429
2,MNE,Montenegro,2002,7.399286
3,MNE,Montenegro,2003,7.303571
4,MNE,Montenegro,2004,7.160714
5,MNE,Montenegro,2005,7.117143
6,MNE,Montenegro,2020,6.305714


In [51]:
# Объединим данные для Черногории с основной таблицей
data = pd.concat([
    data,
    montenegro_data,
], ignore_index=True)

In [52]:
# Удалим столбец Location, так как он больше не нужен.
data.drop(columns=['Location'], inplace=True)

In [53]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_alcohol_prepared.csv', index=False)